In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


###  Query to find out which genres sell the most tracks in the USA

In [3]:

%%sql
WITH usa AS(
    SELECT * FROM invoice
    WHERE billing_country = "USA"
    ),
    usa_tracks AS(
    SELECT *
    FROM usa u
    INNER JOIN invoice_line il ON il.invoice_id = u.invoice_id
    )

SELECT 
    g.name genre, 
    SUM(il.quantity) Quantity_sold, 
    ROUND(SUM(u.total),2) total_sales,
    ROUND((SUM(il.quantity) * 100.0 /
    (
        SELECT SUM(usa_tracks.quantity) from usa_tracks
    )),2) percentage_sold
    
FROM usa u
INNER JOIN invoice_line il ON u.invoice_id = il.invoice_id
INNER JOIN track t ON il.track_id = t.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC

Done.


genre,Quantity_sold,total_sales,percentage_sold
Rock,561,5568.75,53.38
Alternative & Punk,130,1234.53,12.37
Metal,124,1025.64,11.8
R&B/Soul,53,633.6,5.04
Blues,36,453.42,3.43
Alternative,35,369.27,3.33
Latin,22,162.36,2.09
Pop,22,201.96,2.09
Hip Hop/Rap,20,366.3,1.9
Jazz,14,125.73,1.33


The above SQL Queries used a window function to create two tables : usa and usa_tracks because we only need information about USA. We needed to create a two tablew because we need information about USA and in order to avoid writing too many queries and wasting computation time and CPU, we used window Functions to create tables that can be used for our current queries and we used the tables to extract the informations we need.

### Analysis of the total amount of sales assigned to each sales support agent within the company

In [4]:
%%sql
SELECT e.employee_id , e.first_name || ' ' || e.last_name Employee_name, e.hire_date, COUNT(DISTINCT c.customer_id) No_of_customers, ROUND(SUM(i.total),2) Total_sales
FROM employee e
INNER JOIN customer c ON e.employee_id = c.support_rep_id
INNER JOIN invoice i ON c.customer_id = i.customer_id
GROUP BY 1


Done.


employee_id,Employee_name,hire_date,No_of_customers,Total_sales
3,Jane Peacock,2017-04-01 00:00:00,21,1731.51
4,Margaret Park,2017-05-03 00:00:00,20,1584.0
5,Steve Johnson,2017-10-17 00:00:00,18,1393.92


We extract the employees name, hire date from the Employee's table and we calculated the unique number of customers assigned to each Sales agent and their the total sales they made.

## Data Collation from different Countries

In [5]:
%%sql
WITH countries AS(
    SELECT 
    country,
    CASE
        WHEN COUNT(DISTINCT customer_id) = 1 THEN "other" 
        ELSE country
    END AS Country
    FROM customer
    GROUP BY country
    
)
SELECT co.country Country, SUM(c.customer_id) customers, ROUND(SUM(i.total), 2) Total_sales, ROUND((SUM(i.total)/COUNT(DISTINCT c.customer_id)),2) Avg_sales_per_customer, ROUND((SUM(i.total)/COUNT(DISTINCT i.invoice_id)),2) avg_order
FROM countries co                                                                                    
INNER JOIN customer c ON co.country  = c.country
INNER JOIN invoice i ON c.customer_id = i.customer_id
GROUP BY co.Country
ORDER BY co.Country="other", Avg_sales_per_customer DESC

Done.


Country,customers,Total_sales,Avg_sales_per_customer,avg_order
Czech Republic,162,273.24,136.62,9.11
Ireland,598,114.84,114.84,8.83
Spain,550,98.01,98.01,8.91
Chile,741,97.02,97.02,7.46
Portugal,1002,185.13,92.57,6.38
India,1226,183.15,91.57,8.72
Brazil,570,427.68,85.54,7.01
Germany,1130,334.62,83.66,8.16
United Kingdom,1485,245.52,81.84,8.77
Australia,550,81.18,81.18,8.12


We used our window function to generate a table Country where customers are more than one and if the number of customer is equal to one it should return others or else it should return the country's name. Then we selected our country's name from our new table, the total number of customers in each country, the total values of sales, avaerage value of sales per customer and average order per value. 

In [7]:
%%sql
WITH invoice_track AS(
SELECT il.invoice_id, t.album_id
FROM invoice_line il
INNER JOIN track t ON il.track_id = t.track_id
GROUP BY 1
),
check_album AS(
    SELECT it.invoice_id, it.album_id,
    CASE
    WHEN(
        (
        SELECT t.track_id 
        FROM track t 
        INNER JOIN invoice_line il ON t.track_id = il.track_id
        WHERE t.album_id = it.album_id
        GROUP BY t.track_id
        EXCEPT 
        SELECT t.track_id 
        FROM track t
        INNER JOIN invoice_line il ON t.track_id = il.track_id
        WHERE il.invoice_id = it.invoice_id
        GROUP BY t.track_id)IS NULL
        AND
        (
        SELECT t.track_id 
        FROM track t
        INNER JOIN invoice_line il ON t.track_id = il.track_id
        WHERE il.invoice_id = it.invoice_id
        GROUP BY t.track_id
        EXCEPT
        SELECT t.track_id
        FROM track t
        INNER JOIN invoice_line il ON t.track_id = il.track_id
        WHERE t.album_id = it.album_id
        GROUP BY t.track_id) IS NULL
        )
        THEN 1
        ELSE 0
    END AS bought_album
    FROM invoice_track it
    )
SELECT 
COUNT(*) Total_invoice, TOTAL(bought_album) Album_purchased, ROUND((100.0 * TOTAL(bought_album))/COUNT(*), 2) Percentage_invoice
FROM check_album

Done.


Total_invoice,Album_purchased,Percentage_invoice
614,114.0,18.57
